# 第7章 評価履歴の次元削減

https://recsyslab.github.io/recsys-python/ja/chap07.html

## 準備

次のコードを書きなさい。

In [1]:
import numpy as np
import numpy.linalg as LA
np.set_printoptions(precision=3)

# 縮約後の次元数
DIM = 2

Du = np.array([
               [5, 3, 3, +1],
               [6, 2, 5, +1],
               [4, 1, 5, +1],
               [8, 5, 9, -1],
               [2, 4, 2, -1],
               [3, 6, 5, -1],
               [7, 6, 8, -1],
               [4, 2, 3, np.nan],
               [5, 1, 8, np.nan],
               [8, 6, 6, np.nan],
               [3, 4, 2, np.nan],
               [4, 7, 5, np.nan],
               [4, 4, 4, np.nan],
])
I = np.arange(Du.shape[0])
x = Du[:,:-1]
ru = Du[:,-1]

# 分散共分散行列

特徴量$k$の分散$s_{k}^{2}$は次式で求められる。

$$
s_{k}^{2} = \frac{1}{\mid I \mid} \sum_{i \in I} (x_{i,k} - \overline{x}_{k})^{2}
$$

ここで、$\overline{x}_{k}$は特徴量$k$の平均値である。

特徴量$x_{i,k}$を標準化した値$x_{i,k}^{'}$は次式で求められる。

$$
x_{i,k}^{'} = \frac{x_{i,k} - \overline{x}_{k}}{s_{k}}
$$

標準化された特徴量$k$と特徴量$l$の共分散$s_{k,l}$は次式で求められる。

$$
s_{k,l} = \frac{1}{\mid I \mid} \sum_{i \in I} x_{i,k}^{'} x_{i,l}^{'}
$$

各特徴量について求めた分散、共分散をまとめると、次式のように分散共分散行列$\boldsymbol{S}$が得られる。

$$
\boldsymbol{S} = \left[
    \begin{array}{rrrrrr}
         1.000 & 0.191 & 0.749 \\
         0.191 & 1.000 & 0.163 \\
         0.749 & 0.163 & 1.000
    \end{array}
\right]   
$$

このとき、次の問いに答えなさい。

## 01 各特徴量の平均値

`x`において各特徴量の平均値$\overline{x}_{k}$を`ndarray`としてまとめて求めるコードを書きなさい。得られた`ndarray`を`xk_mean`とすること。

In [2]:
xk_mean = np.mean(x, axis=0)
print('xk_mean = {}'.format(xk_mean))

xk_mean = [4.846 3.923 5.   ]


## 02 各特徴量の分散

`x`において各特徴量の分散$s_{k}^{2}$を`ndarray`としてまとめて求めるコードを書きなさい。得られた`ndarray`を`s2`とすること。

In [3]:
s2 = np.var(x, axis=0)
print('s^2 = {}'.format(s2))

s^2 = [3.361 3.763 4.769]


## 03 各特徴量の標準化

`x`において各特徴量を`ndarray`としてまとめて標準化するコードを書きなさい。得られた`ndarray`を`x2`とすること。

In [4]:
# 二重リスト内包表記
# x2 = np.array([[(x[i][j] - xk_mean[j]) / np.sqrt(s2[j]) for j in range(x.shape[1])] for i in I])

# リスト内包表記
# x2 = np.array([(x[i] - xk_mean) / np.sqrt(s2) for i in I])

# リスト内包表記を使わない場合
x2 = (x - xk_mean) / np.sqrt(s2)
print('x\' = \n{}'.format(x2))

x' = 
[[ 0.084 -0.476 -0.916]
 [ 0.629 -0.991  0.   ]
 [-0.462 -1.507  0.   ]
 [ 1.72   0.555  1.832]
 [-1.552  0.04  -1.374]
 [-1.007  1.071  0.   ]
 [ 1.175  1.071  1.374]
 [-0.462 -0.991 -0.916]
 [ 0.084 -1.507  1.374]
 [ 1.72   1.071  0.458]
 [-1.007  0.04  -1.374]
 [-0.462  1.586  0.   ]
 [-0.462  0.04  -0.458]]


## 04 標準化された特徴量kと特徴量lの共分散

標準化された特徴量$k$と特徴量$l$の共分散$s_{k,l}$を求めるコードを書きなさい。得られた値を`skl`とすること。

In [5]:
k = 0
l = 1
skl = np.cov(x2[:,k], x2[:,l], bias=True)[0,1]
print('s{}{} = {:.3f}'.format(k, l, skl))

s01 = 0.191


## 05 分散共分散行列

分散共分散行列$\boldsymbol{S}$を`ndarray`として求めるコードを書きなさい。得られた`ndarray`を`S`とすること。

In [6]:
S = np.cov(x2, rowvar=False, bias=True)
print('S = \n{}'.format(S))

S = 
[[1.    0.191 0.749]
 [0.191 1.    0.163]
 [0.749 0.163 1.   ]]


# 固有値・固有ベクトル

分散共分散行列$\boldsymbol{S}$に対して、

$$
\boldsymbol{S} \boldsymbol{v} = \lambda \boldsymbol{v} \;\;\;\; (\boldsymbol{x} \neq \boldsymbol{0})
$$

を満たす$d$次元ベクトル$\boldsymbol{v}$と実数$\lambda$が存在するとき、$\lambda$を行列$\boldsymbol{S}$の固有値，$\boldsymbol{v}$を$\lambda$に関する行列$\boldsymbol{S}$の固有ベクトルという。このとき、次の問いに答えなさい。


## 06 固有値・固有ベクトル

分散共分散行列$\boldsymbol{S}$の固有値$\lambda$、固有ベクトル$\boldsymbol{v}$を求めるコードを書きなさい。`ndarray`として得られた固有値、固有ベクトルを、それぞれ`lmd`、`v`とすること。

In [7]:
lmd, v = np.linalg.eig(S)
print('λ = {}'.format(lmd))
print('v = \n{}'.format(v))

λ = [1.826 0.25  0.924]
v = 
[[-0.679 -0.71   0.186]
 [-0.291  0.028 -0.956]
 [-0.674  0.704  0.225]]


## 07 固有値の降順にソートしたインデックス配列

固有値`lmd`について、降順にソートしたインデックスの配列を`ndarray`として生成するコードを書きなさい。得られた`ndarray`を`indices`とすること。

In [8]:
indices = np.argsort(lmd)[::-1]
print('indices = {}'.format(indices))

indices = [0 2 1]


## 08 固有値の降順に固有値配列をソート

固有値の降順にソートした固有値配列を`ndarray`として生成するコードを書きなさい。得られた`ndarray`を`lmd`とすること。

In [9]:
lmd = lmd[indices]
print('λ = {}'.format(lmd))

λ = [1.826 0.924 0.25 ]


## 09 固有値の降順に固有ベクトル配列をソート

固有値の降順にソートした固有ベクトル配列を`ndarray`として生成するコードを書きなさい。得られた`ndarray`を`v`とすること。

In [10]:
v = v[:,indices]
print('v = \n{}'.format(v))

v = 
[[-0.679  0.186 -0.71 ]
 [-0.291 -0.956  0.028]
 [-0.674  0.225  0.704]]


## 10 第d主成分までの固有ベクトル

第`DIM`主成分までの対応する固有ベクトルを列ベクトルとして並べた行列$\boldsymbol{V}$を`ndarray`として生成するコードを書きなさい。得られた`ndarray`を`V`とすること。

In [11]:
V = v[:,:DIM]
print('V = \n{}'.format(V))

V = 
[[-0.679  0.186]
 [-0.291 -0.956]
 [-0.674  0.225]]


# 主成分得点

アイテム$i$の第$k$主成分得点$x_{i,k}^{''}$は次式で求められる。

$$
x_{i,k}^{''} = \sum_{l = 1}^{d} x_{i,l}^{'} v_{k,l}
$$

アイテム$i$の次元削減後の特徴ベクトル$\boldsymbol{x}_{i}^{''}$は次式で求められる。

$$
\boldsymbol{x}_{i}^{''\mathsf{T}} = \boldsymbol{x}_{i}^{'\mathsf{T}} \boldsymbol{V}
$$

このとき、次の問いに答えなさい。

## 11 アイテムiの第k主成分得点

アイテム$i$の第$k$主成分得点$x_{i,k}^{''}$を求めるコードを書きなさい。得られた値を`xik3`とすること。

In [23]:
i = 0
k = 0
xik3 = np.sum([x2[i,l] * V[l,k] for l in range(x2[0].size)])
print('x{}{}\'\' = {:.3f}'.format(i, k, xik3))

x00'' = 0.699


## 12 各アイテムの次元削減後の特徴ベクトル

各アイテムの次元削減後の特徴ベクトルを`ndarray`としてまとめて求めるコードを書きなさい。得られた`ndarray`を`x3`とすること。

In [25]:
x3 = x2@V
print('x\'\' = \n{}'.format(x3))

x'' = 
[[ 0.699  0.264]
 [-0.139  1.065]
 [ 0.752  1.355]
 [-2.564  0.202]
 [ 1.969 -0.636]
 [ 0.373 -1.211]
 [-2.035 -0.496]
 [ 1.219  0.656]
 [-0.545  1.766]
 [-1.788 -0.601]
 [ 1.598 -0.535]
 [-0.148 -1.603]
 [ 0.611 -0.227]]


# 寄与率

第$k$主成分の寄与率は次式で求められる。

$$
\frac{\lambda_{k}}{\sum_{l=1}^{d} \lambda_{l}}
$$

第$k$主成分までの累積寄与率は次式で求められる。

$$
\frac{\sum_{l=1}^{k} \lambda_{l}}{\sum_{l=1}^{d} \lambda_{l}}
$$

このとき、次の問いに答えなさい。

## 13 第k主成分の寄与率

第$k$主成分の寄与率を求めるコードを書きなさい。得られた値を`pk`とすること。

In [26]:
k = 0
pk = lmd[k] / np.sum(lmd)
print('第{}主成分の寄与率 = {:.3f}'.format(k+1, pk))

第1主成分の寄与率 = 0.609


## 14 第k主成分までの累積寄与率

第$k$主成分までの累積寄与率を求めるコードを書きなさい。得られた値を`ck`とすること。

In [28]:
k = 2
ck = np.sum(lmd[:k]) / np.sum(lmd)
print('第{}主成分までの累積寄与率 = {:.3f}'.format(k, ck))

第2主成分までの累積寄与率 = 0.917


# 推薦

次元削減後の評価履歴$D_{u}^{'}$は、次式のとおり、次元削減後の特徴ベクトル$\boldsymbol{x}_{i}^{''}$と元の評価値$r_{u,i}$を結合することで得られる。

$$
D_{u}^{'} = \left[
 \begin{array}{rrr}
   0.699 &  0.264 & +1 \\
  -0.139 &  1.065 & +1 \\
   0.752 &  1.355 & +1 \\
  -2.564 &  0.202 & -1 \\
   1.969 & -0.636 & -1 \\
   0.373 & -1.211 & -1 \\
  -2.035 & -0.496 & -1 \\
   1.219 &  0.656 &  ? \\
  -0.545 &  1.766 &  ? \\
  -1.788 & -0.601 &  ? \\
   1.598 & -0.535 &  ? \\
  -0.148 & -1.603 &  ? \\
   0.611 & -0.227 &  ?
 \end{array}
\right]
$$

このとき、次の問いに答えなさい。

## 15 次元削減後の評価履歴

次元削減後の評価履歴$D_{u}^{'}$を`ndarray`として生成するコードを書きなさい。得られた`ndarray`を`Du2`とすること。

In [29]:
Du2 = np.c_[x3, ru]
print('R\' = \n{}'.format(Du2))

R' = 
[[ 0.699  0.264  1.   ]
 [-0.139  1.065  1.   ]
 [ 0.752  1.355  1.   ]
 [-2.564  0.202 -1.   ]
 [ 1.969 -0.636 -1.   ]
 [ 0.373 -1.211 -1.   ]
 [-2.035 -0.496 -1.   ]
 [ 1.219  0.656    nan]
 [-0.545  1.766    nan]
 [-1.788 -0.601    nan]
 [ 1.598 -0.535    nan]
 [-0.148 -1.603    nan]
 [ 0.611 -0.227    nan]]
